In [1]:
import torch
from torch.utils.data import DataLoader
from state_quantization.dataset import load_dataset
from state_quantization.dataset import DynamicsModelDataset
from state_quantization.train import train_model, test_step
from state_quantization.eval import eval_model, compare_models
from state_quantization.forcasting_models import LSTMForcasting, LSTMForcastingConstOut
from state_quantization.trainer import NNTrainer
from torch.optim.lr_scheduler import MultiStepLR

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True

Using Device: cuda:0


## Load Dataset

In [3]:
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples-la.npy'
normalized_data_params_save_path = 'tmp/transformer/NormalizeInputConfigs.pkl'
dataset_device = 'cpu'
y_indexes = [4, 6]

train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key,
                                          output_key=dataset_output_key, dataset_class=DynamicsModelDataset,
                                          normalize=True, device=dataset_device, y_clip_range=y_indexes,
                                          normalized_input_data_params_save_path=normalized_data_params_save_path)

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last': True,
          'pin_memory': not train_dataset.x.is_cuda}
print(params)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)
train_dataset.y[0]

Clipping y
torch.Size([672000, 39, 6])
torch.Size([672000, 10, 2])
torch.Size([288000, 39, 6])
torch.Size([288000, 10, 2])
{'batch_size': 8000, 'shuffle': True, 'num_workers': 0, 'drop_last': True, 'pin_memory': True}


tensor([[-0.8233,  0.3537],
        [-0.6556,  0.2818],
        [-0.8421,  0.2420],
        [-0.8779,  0.2863],
        [-0.6942,  0.2825],
        [-0.8032,  0.2881],
        [-0.7378,  0.2211],
        [-0.6053,  0.1703],
        [-0.4699,  0.1287],
        [-0.5391,  0.1608]])

## Define model

In [4]:
model_dict_path = 'state_quantization/model_no_disc_dict'
model_path = 'state_quantization/model_no_disc'
num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 20
out_size = train_dataset.get_output_feature_size()
print(f'Out Size:{out_size}')
look_ahead = train_dataset.get_look_ahead_size()
n_layers = 1
dropout = 0.1
print(look_ahead)
model = forcasting_model = LSTMForcasting(features=num_of_features, hidden_size=hidden_size, out_size=out_size, seq_len=seq_len,
                                  look_ahead=look_ahead, dropout=dropout, n_layers=n_layers).to(device=device)

load_to_gpu = model.is_cuda() and not train_dataset.x.is_cuda
print(load_to_gpu)

Out Size:2
9
LSTM Layers
ModuleList(
  (0): LSTMCell(6, 20)
)
LSTM Dropout Layers
ModuleList()
Fully Connected Layers
Sequential(
  (0): Linear(in_features=20, out_features=20, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=20, out_features=20, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=20, out_features=2, bias=True)
)
True


## Train Model

In [5]:


lr = 1e-3

lr_milestones=[24, 54]
gamma = 0.1

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = MultiStepLR(optimizer, milestones=lr_milestones, gamma=gamma)
trainer = NNTrainer(model=model,train_loader=train_loader, test_loader=val_loader,load_to_gpu=load_to_gpu,optimizer=optimizer, lr_scheduler=lr_scheduler)

In [6]:
trainer.train(n_epochs=50)
# train_model(model=model, train_loader=train_loader, test_loader=val_loader, n_epochs=100, learning_rate=1e-3,
#             load_to_gpu=load_to_gpu, gamma=0.1, lr_milestones=[25, 200])

Untrained test
--------
--------------------------------------
Model Test loss: 1.0279005186425314
--------------------------------------
--------------------------------------
Epoch 1
---------
--------------------------------------
Model Train loss: 0.7831801176071167
--------------------------------------
Model Test loss: 0.3615417066547606
--------------------------------------
lr: [0.001]
Epoch time: epoch_time = 8.980s
--------------------------------------
--------------------------------------
Epoch 2
---------
--------------------------------------
Model Train loss: 0.19947873889690354
--------------------------------------
Model Test loss: 0.08821214963164595
--------------------------------------
lr: [0.001]
Epoch time: epoch_time = 8.951s
--------------------------------------
--------------------------------------
Epoch 3
---------
--------------------------------------
Model Train loss: 0.10570650982360046
--------------------------------------
Model Test loss: 0.06302970

## Save Model

In [7]:

torch.save(model.state_dict(), model_dict_path)
#torch.save(model, model_path)
torch.cuda.empty_cache()
del model

## Load Model for evaluation

In [8]:

model_classes = [LSTMForcasting, LSTMForcastingConstOut]
models = []
for model_class in model_classes:
    models.append(model_class(features=num_of_features, hidden_size=hidden_size, out_size=out_size,
                              seq_len=seq_len, n_layers=n_layers, dropout_p=dropout,
                              look_ahead=look_ahead).to(device))
    models[-1].load_state_dict(torch.load(model_dict_path))

TypeError: __init__() got an unexpected keyword argument 'dropout_p'

In [ ]:
for la_model in models:
    print(la_model.__class__.__name__)
    test_step(model=la_model, data_loader=val_loader, loss_function=torch.nn.MSELoss(), load_to_gpu=load_to_gpu)

In [ ]:
#value_keys = ['setpoint', 'velocity', 'gain', 'shift', 'fatigue', 'consumption']
value_keys = ['fatigue', 'consumption']

models = []
for model_class in model_classes:
    models.append(model_class(features=num_of_features, hidden_size=hidden_size, out_size=out_size,
                              seq_len=seq_len, n_layers=n_layers, dropout_p=dropout,
                              look_ahead=look_ahead).to(dataset_device))
    models[-1].load_state_dict(torch.load(model_dict_path))

compare_models(models, x=val_dataset.x[78], y=val_dataset.y[78], value_keys=value_keys)